In [1]:
import os
import cv2
import numpy as np
import random

# Function to preprocess images
def preprocess_images(input_dir, output_dir):
    # Create output directories if they don't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if not os.path.exists(os.path.join(output_dir, 'Train')):
        os.makedirs(os.path.join(output_dir, 'Train'))
    if not os.path.exists(os.path.join(output_dir, 'Train', 'pos')):
        os.makedirs(os.path.join(output_dir, 'Train', 'pos'))
    if not os.path.exists(os.path.join(output_dir, 'Train', 'neg')):
        os.makedirs(os.path.join(output_dir, 'Train', 'neg'))
    if not os.path.exists(os.path.join(output_dir, 'Test')):
        os.makedirs(os.path.join(output_dir, 'Test'))
    if not os.path.exists(os.path.join(output_dir, 'Test', 'pos')):
        os.makedirs(os.path.join(output_dir, 'Test', 'pos'))
    if not os.path.exists(os.path.join(output_dir, 'Test', 'neg')):
        os.makedirs(os.path.join(output_dir, 'Test', 'neg'))
    
    # Process training positive samples
    for filename in os.listdir(os.path.join(input_dir, 'Train', 'pos')):
        img_path = os.path.join(input_dir, 'Train', 'pos', filename)
        img = cv2.imread(img_path)
        # Resize image to a fixed size (e.g., 128x64)
        img = cv2.resize(img, (128, 64))
        # Convert image to grayscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Apply median filter for noise reduction
        img = cv2.medianBlur(img, 3)
        # Data augmentation: horizontal flip
        if random.random() > 0.5:
            img = cv2.flip(img, 1)
        # Save preprocessed image
        cv2.imwrite(os.path.join(output_dir, 'Train', 'pos', filename), img)
    
    # Process training negative samples
    for filename in os.listdir(os.path.join(input_dir, 'Train', 'neg')):
        img_path = os.path.join(input_dir, 'Train', 'neg', filename)
        img = cv2.imread(img_path)
        # Resize image to a fixed size (e.g., 128x64)
        img = cv2.resize(img, (128, 64))
        # Convert image to grayscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Apply median filter for noise reduction
        img = cv2.medianBlur(img, 3)
        # Data augmentation: horizontal flip
        if random.random() > 0.5:
            img = cv2.flip(img, 1)
        # Save preprocessed image
        cv2.imwrite(os.path.join(output_dir, 'Train', 'neg', filename), img)

    # Process testing positive samples
    for filename in os.listdir(os.path.join(input_dir, 'Test', 'pos')):
        img_path = os.path.join(input_dir, 'Test', 'pos', filename)
        img = cv2.imread(img_path)
        # Resize image to a fixed size (e.g., 128x64)
        img = cv2.resize(img, (128, 64))
        # Convert image to grayscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Apply median filter for noise reduction
        img = cv2.medianBlur(img, 3)
        # Save preprocessed image
        cv2.imwrite(os.path.join(output_dir, 'Test', 'pos', filename), img)
    
    # Process testing negative samples
    for filename in os.listdir(os.path.join(input_dir, 'Test', 'neg')):
        img_path = os.path.join(input_dir, 'Test', 'neg', filename)
        img = cv2.imread(img_path)
        # Resize image to a fixed size (e.g., 128x64)
        img = cv2.resize(img, (128, 64))
        # Convert image to grayscale
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Apply median filter for noise reduction
        img = cv2.medianBlur(img, 3)
        # Save preprocessed image
        cv2.imwrite(os.path.join(output_dir, 'Test', 'neg', filename), img)

# Define input and output directories
input_dir = 'INRIAPerson'
output_dir = 'Preprocessed_Data'

# Perform preprocessing
preprocess_images(input_dir, output_dir)


In [2]:
import cv2
import numpy as np
from skimage.feature import hog, local_binary_pattern
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import os


# Function to load images and labels
def load_images_from_folder(folder):
    images = []
    labels = []
    pos_folder = os.path.join(folder, 'Train', 'pos')
    neg_folder = os.path.join(folder, 'Train', 'neg')
    
    for filename in os.listdir(pos_folder):
        img = cv2.imread(os.path.join(pos_folder, filename))
        if img is not None:
            images.append(img)
            labels.append(1)  # Positive class

    for filename in os.listdir(neg_folder):
        img = cv2.imread(os.path.join(neg_folder, filename))
        if img is not None:
            images.append(img)
            labels.append(0)  # Negative class

    return images, labels



# Load datasets from Preprocessed_Data folder
images, labels = load_images_from_folder('Preprocessed_Data')

# Function to extract HOG features
def extract_hog_features(images):
    hog_features = []
    for img in images:
        img = cv2.resize(img, (64, 128))  # Resize to standard dimensions
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        f = hog(gray, orientations=9, pixels_per_cell=(8, 8),
                cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False, feature_vector=True)
        hog_features.append(f)
    
    print(f"Number of images: {len(images)}")
    print(f"Number of HOG features extracted from the first image: {len(hog_features[0])}")
    
    return np.array(hog_features)

# Function to extract LBP features
def extract_lbp_features(images):
    lbp_features = []
    for img in images:
        img = cv2.resize(img, (64, 128))  # Resize to standard dimensions
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        lbp = local_binary_pattern(gray, 24, 8, method='uniform')
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 24 + 3), range=(0, 24 + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-6)
        lbp_features.append(hist)
    return np.array(lbp_features)

# Function to extract LTP features
def extract_ltp_features(images):
    ltp_features = []
    for img in images:
        img = cv2.resize(img, (64, 128))  # Resize to standard dimensions
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ltp_up = np.where((gray - cv2.GaussianBlur(gray, (5, 5), 0)) >= 0.05, 1, 0)
        ltp_down = np.where((gray - cv2.GaussianBlur(gray, (5, 5), 0)) <= -0.05, 1, 0)
        ltp_feat = np.hstack((ltp_up.ravel(), ltp_down.ravel()))
        ltp_features.append(ltp_feat)
    return np.array(ltp_features)

# Combine HOG and LBP features
def combine_features(images):
    hog_features = extract_hog_features(images)
    lbp_features = extract_lbp_features(images)
    ltp_features = extract_ltp_features(images)
    
    print(f"Number of HOG features: {hog_features.shape[1]}")
    print(f"Number of LBP features: {lbp_features.shape[1]}")
    print(f"Number of LTP features: {ltp_features.shape[1]}")
    
    combined_features = np.hstack((hog_features, lbp_features, ltp_features))
    return combined_features

# Extract combined features
features = combine_features(images)

# Normalize features
scaler = StandardScaler()
features_normalized = scaler.fit_transform(features)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(features_normalized, labels, test_size=0.2, random_state=0)

# Create and train the SVM with an RBF kernel
classifier = svm.SVC(kernel='rbf', C=10, gamma='scale', probability=True)
classifier.fit(X_train, y_train)

# Evaluate the model
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")



Number of images: 1832
Number of HOG features extracted from the first image: 3780
Number of HOG features: 3780
Number of LBP features: 26
Number of LTP features: 16384
Accuracy: 85.83%


In [3]:
import cv2
import numpy as np
from skimage.feature import hog, local_binary_pattern
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import os

# Function to load images and labels
def load_images_from_folder(folder):
    images = []
    labels = []
    pos_folder = os.path.join(folder, 'Train', 'pos')
    neg_folder = os.path.join(folder, 'Train', 'neg')
    
    for filename in os.listdir(pos_folder):
        img = cv2.imread(os.path.join(pos_folder, filename))
        if img is not None:
            images.append(img)
            labels.append(1)  # Positive class

    for filename in os.listdir(neg_folder):
        img = cv2.imread(os.path.join(neg_folder, filename))
        if img is not None:
            images.append(img)
            labels.append(0)  # Negative class

    return images, labels

# Load datasets from Preprocessed_Data folder
images, labels = load_images_from_folder('Preprocessed_Data')

# Function to extract HOG features
def extract_hog_features(images):
    hog_features = []
    for img in images:
        img = cv2.resize(img, (64, 128))  # Resize to standard dimensions
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        f = hog(gray, orientations=9, pixels_per_cell=(8, 8),
                cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False, feature_vector=True)
        hog_features.append(f)
    
    print(f"Number of images: {len(images)}")
    print(f"Number of HOG features extracted from the first image: {len(hog_features[0])}")
    
    return np.array(hog_features)

# Function to extract LBP features
def extract_lbp_features(images):
    lbp_features = []
    for img in images:
        img = cv2.resize(img, (64, 128))  # Resize to standard dimensions
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        lbp = local_binary_pattern(gray, 24, 8, method='uniform')
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 24 + 3), range=(0, 24 + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-6)
        lbp_features.append(hist)
    return np.array(lbp_features)

# Function to extract LTP features
def extract_ltp_features(images):
    ltp_features = []
    for img in images:
        img = cv2.resize(img, (64, 128))  # Resize to standard dimensions
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ltp_up = np.where((gray - cv2.GaussianBlur(gray, (5, 5), 0)) >= 0.05, 1, 0)
        ltp_down = np.where((gray - cv2.GaussianBlur(gray, (5, 5), 0)) <= -0.05, 1, 0)
        ltp_feat = np.hstack((ltp_up.ravel(), ltp_down.ravel()))
        ltp_features.append(ltp_feat)
    return np.array(ltp_features)

# Combine HOG and LBP features
def combine_features(images):
    hog_features = extract_hog_features(images)
    lbp_features = extract_lbp_features(images)
    ltp_features = extract_ltp_features(images)
    
    print(f"Number of HOG features: {hog_features.shape[1]}")
    print(f"Number of LBP features: {lbp_features.shape[1]}")
    print(f"Number of LTP features: {ltp_features.shape[1]}")
    
    combined_features = np.hstack((hog_features, lbp_features, ltp_features))
    return combined_features

# Extract combined features
features = combine_features(images)

# Data augmentation (manually)
augmented_features = []
augmented_labels = []

for feature, label in zip(features, labels):
    augmented_features.append(feature)
    augmented_labels.append(label)
    
    # Augment feature by adding random noise (example augmentation)
    augmented_feature = feature + np.random.normal(loc=0, scale=0.01, size=feature.shape)
    augmented_features.append(augmented_feature)
    augmented_labels.append(label)

# Convert augmented data to numpy arrays
augmented_features = np.array(augmented_features)
augmented_labels = np.array(augmented_labels)

# Normalize features
scaler = StandardScaler()
features_normalized = scaler.fit_transform(augmented_features)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(features_normalized, augmented_labels, test_size=0.2, random_state=0)

# Create and train the SVM with an RBF kernel
classifier = svm.SVC(kernel='rbf', C=10, gamma='scale', probability=True)
classifier.fit(X_train, y_train)

# Evaluate the model
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


Number of images: 1832
Number of HOG features extracted from the first image: 3780
Number of HOG features: 3780
Number of LBP features: 26
Number of LTP features: 16384
Accuracy: 92.91%


In [4]:
from sklearn.metrics import classification_report

# Evaluate the model
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Calculate additional metrics
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 92.91%
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.97      0.95       466
           1       0.95      0.85      0.90       267

    accuracy                           0.93       733
   macro avg       0.93      0.91      0.92       733
weighted avg       0.93      0.93      0.93       733



In [5]:
# Calculate the mentioned metrics
detected_target = sum(y_pred)  # Number of positive predictions
false_detections = sum((y_pred == 1) & (y_test == 0))  # False positive predictions
missed_detections = sum((y_pred == 0) & (y_test == 1))  # False negative predictions
semi_accurate_number = sum(y_pred == y_test)  # Number of semi-accurate predictions
accurate_number = sum((y_pred == 1) & (y_test == 1))  # True positive predictions

# Calculate rates
false_detection_rate = false_detections / sum(y_test == 0)
missed_detection_rate = missed_detections / sum(y_test == 1)
semi_accuracy_rate = semi_accurate_number / len(y_pred)


# Calculate total
total = len(y_pred)

# Print the results
print(f"Detected target: {detected_target}")
print(f"False detections: {false_detections}")
print(f"Missed detections: {missed_detections}")
print(f"Semi-accurate number: {semi_accurate_number}")
print(f"Accurate number: {accurate_number}")
print(f"False detection rate: {false_detection_rate:.2f}")
print(f"Missed detection rate: {missed_detection_rate:.2f}")
print(f"Semi-accuracy rate: {semi_accuracy_rate:.2f}")
print(f"Total: {total}")


Detected target: 241
False detections: 13
Missed detections: 39
Semi-accurate number: 681
Accurate number: 228
False detection rate: 0.03
Missed detection rate: 0.15
Semi-accuracy rate: 0.93
Total: 733


In [6]:
from sklearn.metrics import confusion_matrix

# Assuming y_test contains the true labels and y_pred contains the predicted labels
conf_matrix = confusion_matrix(y_test, y_pred)

# Print the confusion matrix
print("Confusion Matrix:")
print(conf_matrix)


Confusion Matrix:
[[453  13]
 [ 39 228]]


In [7]:
from sklearn.metrics import confusion_matrix

# Assuming y_true contains the true labels and y_pred contains the predicted labels
# Compute the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Extract true positives (TP), false negatives (FN), true negatives (TN), and false positives (FP)
TP = cm[1, 1]
FN = cm[1, 0]

# Calculate miss rate (false negative rate)
miss_rate = (FN / (TP + FN)) * 100

print(f"Miss rate: {miss_rate:.2f}%")


Miss rate: 14.61%


In [8]:
from sklearn.metrics import confusion_matrix

# Assuming y_true contains the true labels and y_pred contains the predicted labels
# Compute the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Extract true positives (TP), false negatives (FN), true negatives (TN), and false positives (FP)
TP = cm[1, 1]
FN = cm[1, 0]

# Calculate missed detection rate (false negative rate)
missed_detection_rate = (FN / (TP + FN)) * 100

print(f"Missed detection rate: {missed_detection_rate:.2f}%")


Missed detection rate: 14.61%


In [9]:
# Calculate Total number of detections

from sklearn.metrics import confusion_matrix

# Assuming y_true contains the true labels and y_pred contains the predicted labels
# Compute the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Extract true positives (TP), false negatives (FN), true negatives (TN), and false positives (FP)
TP = cm[1, 1]
FN = cm[1, 0]

#total_detections = false_detection_number + missed_detection_number + semi_accurate_number

total_detections = FN + 45 + 674

print("Total detections:", total_detections)


Total detections: 758


In [ ]:
import cv2
import numpy as np
from skimage.feature import hog, local_binary_pattern
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
import os

# Function to load images and labels
def load_images_from_folder(folder):
    images = []
    labels = []
    pos_folder = os.path.join(folder, 'Train', 'pos')
    neg_folder = os.path.join(folder, 'Train', 'neg')
    
    for filename in os.listdir(pos_folder):
        img = cv2.imread(os.path.join(pos_folder, filename))
        if img is not None:
            images.append(img)
            labels.append(1)  # Positive class

    for filename in os.listdir(neg_folder):
        img = cv2.imread(os.path.join(neg_folder, filename))
        if img is not None:
            images.append(img)
            labels.append(0)  # Negative class

    return images, labels

# Load datasets from Preprocessed_Data folder
images, labels = load_images_from_folder('Preprocessed_Data')

# Function to extract HOG features
def extract_hog_features(images):
    hog_features = []
    for img in images:
        img = cv2.resize(img, (64, 128))  # Resize to standard dimensions
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        f = hog(gray, orientations=9, pixels_per_cell=(8, 8),
                cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False, feature_vector=True)
        hog_features.append(f)
    
    return np.array(hog_features)

# Function to extract LBP features
def extract_lbp_features(images):
    lbp_features = []
    for img in images:
        img = cv2.resize(img, (64, 128))  # Resize to standard dimensions
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        lbp = local_binary_pattern(gray, 24, 8, method='uniform')
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 24 + 3), range=(0, 24 + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-6)
        lbp_features.append(hist)
    return np.array(lbp_features)

# Function to extract LTP features
def extract_ltp_features(images):
    ltp_features = []
    for img in images:
        img = cv2.resize(img, (64, 128))  # Resize to standard dimensions
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ltp_up = np.where((gray - cv2.GaussianBlur(gray, (5, 5), 0)) >= 0.05, 1, 0)
        ltp_down = np.where((gray - cv2.GaussianBlur(gray, (5, 5), 0)) <= -0.05, 1, 0)
        ltp_feat = np.hstack((ltp_up.ravel(), ltp_down.ravel()))
        ltp_features.append(ltp_feat)
    return np.array(ltp_features)

# Combine HOG, LBP, and LTP features
def combine_features(images):
    hog_features = extract_hog_features(images)
    lbp_features = extract_lbp_features(images)
    ltp_features = extract_ltp_features(images)
    
    combined_features = np.hstack((hog_features, lbp_features, ltp_features))
    return combined_features

# Extract combined features
features = combine_features(images)

# Data augmentation
augmented_features = []
augmented_labels = []

for feature, label in zip(features, labels):
    augmented_features.append(feature)
    augmented_labels.append(label)
    
    # Augment feature by adding random noise (example augmentation)
    augmented_feature = feature + np.random.normal(loc=0, scale=0.01, size=feature.shape)
    augmented_features.append(augmented_feature)
    augmented_labels.append(label)

# Convert augmented data to numpy arrays
augmented_features = np.array(augmented_features)
augmented_labels = np.array(augmented_labels)

# Normalize features
scaler = StandardScaler()
features_normalized = scaler.fit_transform(augmented_features)

# Fit GMM with fewer components
gmm = GaussianMixture(n_components=3, covariance_type='full', random_state=0)
gmm.fit(features_normalized)

# Extract GMM parameters as features
gmm_features = gmm.predict_proba(features_normalized)

# Combine GMM features with existing features
combined_features_with_gmm = np.hstack((features_normalized, gmm_features))

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(combined_features_with_gmm, augmented_labels, test_size=0.2, random_state=0)

# Create and train the SVM with an RBF kernel
classifier = svm.SVC(kernel='rbf', C=10, gamma='scale', probability=True)
classifier.fit(X_train, y_train)

# Evaluate the model
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
